<a href="https://colab.research.google.com/github/trueforme/ml4se/blob/main/lecture1/hw1_ml4se_Sharavyev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Модель - GPT5 Thinking, с подпиской PLUS; Не уверен, что было честно использовать эту модель, так как считаю ее довольно мощной, но в правилах ограничения на модель не было.

PROMPT: Задание:
Сгенерируй простую программу на python, реализующую dfs, в файле не должно быть if __name__. Сгенерированный код должен содержать tabs ( \t ) в виде отступов для строк кода, вместо пробелов. Должен содержать лишние \n ( назовем это "шумом"). Затем этот файл нужно перевести в .txt формат, где будут видны все whitespaces символы. Затем следующее подзадание:
Нужно обработать этот  текстовый файл. ИЗМЕНИ ТОЛЬКО ТАК, как я попрошу, и НИКАК ИНАЧЕ:
1. Нужно убрать ВСЕ комментарии
2. ТАБЫ ( tabs ) заменить соответствующим им ПРОБЕЛАМ для таких же отступов
3. ВСЕ НЕЗНАЧАЩИЕ пробельные символы ( whitespaces) убрать
4. ВСЕ названия ПЕРЕМЕННЫХ, ФУНКЦИЙ, КЛАССОВ заменить на name_{old_name} В конце проверь, что ты сделал как требуется, и не добавил ничего лишнего

В итоге должно получиться 4 файла:
1) raw.py, с \t, лишними \n, как я описал
2) raw.txt, переведенный raw.py в txt формат
3) clean.txt, очищенный по правилам подзадания raw.txt
4) clean.py, переведенный clean.txt в питоновский файл, сохраняющий полную работоспособность в точности как в raw.py


Доп Промпт: Все отлично! Единственное замечание, которое нужно исправить: в raw.py, raw.txt в конце строк должны быть лишние ( незначающие) \n. То есть как минимум в кнце каждой строчки вместо \n в конце должно быть \n\n. Остальное задание остается прежним. Переделай.

In [ ]:
# Simple DFS implementation (with tabs and noise)

def dfs(graph, start, visited=None):

	if visited is None:

		visited = set()

	# mark visited

	visited.add(start)

	for neighbor in graph.get(start, []):

		if neighbor not in visited:

			dfs(graph, neighbor, visited)

	return visited

# sample graph

graph = {

	'A': ['B', 'C'],

	'B': ['D'],

	'C': ['E'],

	'D': [],

	'E': []

}

start = 'A'

result = dfs(graph, start)

print(sorted(result))



In [ ]:
def name_dfs(name_graph, name_start, name_visited=None):
    if name_visited is None:
        name_visited = set()
    name_visited.add(name_start)
    for name_neighbor in name_graph.get(name_start, []):
        if name_neighbor not in name_visited:
            name_dfs(name_graph, name_neighbor, name_visited)
    return name_visited
name_graph = {'A': ['B', 'C'], 'B': ['D'], 'C': ['E'], 'D': [], 'E': []}
name_start = 'A'
name_result = name_dfs(name_graph, name_start)
print(sorted(name_result))


Код нейронки:

In [ ]:
# Rebuild the files with the requested change: each line in raw.py should have *two* newlines ("\n\n")
# and raw.txt should display these as visible \\n\\n at the end of each line.
# The rest of the processing (cleaning steps, renaming, etc.) stays the same.

import os, io, re, ast, contextlib, runpy

BASE = "/mnt/data"

# 1) Create raw.py with tabs and double newlines after each line
raw_py_path = os.path.join(BASE, "raw.py")
raw_code_lines = [
    "# Simple DFS implementation (with tabs and noise)",
    "def dfs(graph, start, visited=None):",
    "\tif visited is None:",
    "\t\tvisited = set()",
    "\t# mark visited",
    "\tvisited.add(start)",
    "\tfor neighbor in graph.get(start, []):",
    "\t\tif neighbor not in visited:",
    "\t\t\tdfs(graph, neighbor, visited)",
    "\treturn visited",
    "# sample graph",
    "graph = {",
    "\t'A': ['B', 'C'],",
    "\t'B': ['D'],",
    "\t'C': ['E'],",
    "\t'D': [],",
    "\t'E': []",
    "}",
    "start = 'A'",
    "result = dfs(graph, start)",
    "print(sorted(result))"
]

raw_code = "\n\n".join(raw_code_lines) + "\n\n"  # add double newlines between lines

with open(raw_py_path, "w", encoding="utf-8", newline="") as f:
    f.write(raw_code)

# 2) Produce raw.txt where \t and \n are visible, with double \n shown as \\n\\n
def to_visible_whitespace_double(txt: str) -> str:
    lines = txt.splitlines(keepends=True)
    out_lines = []
    for line in lines:
        # Replace tabs with literal \t
        vis = line.replace("\t", "\\t")
        # Replace single or double newlines with visible \n\n
        count_newlines = len(line) - len(line.rstrip("\n"))
        vis = vis.rstrip("\n") + ("\\n" * count_newlines)
        out_lines.append(vis)
    return "\n".join(out_lines)

raw_txt_path = os.path.join(BASE, "raw.txt")
raw_visible = to_visible_whitespace_double(raw_code)
with open(raw_txt_path, "w", encoding="utf-8", newline="\n") as f:
    f.write(raw_visible)

# 3) Transform raw.txt -> clean.txt / clean.py as before
def from_visible_whitespace(txt: str) -> str:
    lines = txt.splitlines()
    real_lines = []
    for line in lines:
        content = line.replace("\\t", "\t")
        # Replace visible \n\n or single \n
        content = re.sub(r"(\\n){2}", "\n\n", content)
        content = re.sub(r"(?<!\\)\\n", "\n", content)
        real_lines.append(content)
    return "\n".join(real_lines) + "\n"

with open(raw_txt_path, "r", encoding="utf-8") as f:
    raw_txt_content = f.read()

code_from_txt = from_visible_whitespace(raw_txt_content)

def strip_comments(src: str) -> str:
    out_lines = []
    for line in src.splitlines():
        in_single = in_double = escape = False
        cut_index = None
        for i, ch in enumerate(line):
            if escape:
                escape = False
                continue
            if ch == "\\":
                escape = True
                continue
            if ch == "'" and not in_double:
                in_single = not in_single
            elif ch == '"' and not in_single:
                in_double = not in_double
            elif ch == "#" and not in_single and not in_double:
                cut_index = i
                break
        if cut_index is not None:
            out_lines.append(line[:cut_index])
        else:
            out_lines.append(line)
    return "\n".join(out_lines) + "\n"

no_comments = strip_comments(code_from_txt)

def tabs_to_spaces(src: str, tabsize: int = 4) -> str:
    out_lines = []
    for line in src.splitlines():
        leading_tabs = len(line) - len(line.lstrip("\t"))
        prefix = " " * (leading_tabs * tabsize)
        rest = line.lstrip("\t").replace("\t", " " * tabsize)
        out_lines.append(prefix + rest)
    return "\n".join(out_lines) + "\n"

spaces_code = tabs_to_spaces(no_comments, tabsize=4)

def remove_non_significant_ws(src: str) -> str:
    lines = [ln.rstrip(" ") for ln in src.splitlines()]
    lines = [ln for ln in lines if ln.strip() != ""]
    return "\n".join(lines) + "\n"

minified_ws = remove_non_significant_ws(spaces_code)

SAFE_EXCLUSIONS = {"print", "set", "sorted"}

class Renamer(ast.NodeTransformer):
    def __init__(self, mapping):
        self.mapping = mapping
    def visit_FunctionDef(self, node):
        if node.name in self.mapping:
            node.name = self.mapping[node.name]
        self.generic_visit(node)
        return node
    def visit_ClassDef(self, node):
        if node.name in self.mapping:
            node.name = self.mapping[node.name]
        self.generic_visit(node)
        return node
    def visit_Name(self, node):
        if node.id in self.mapping:
            node.id = self.mapping[node.id]
        return node
    def visit_arg(self, node):
        if node.arg in self.mapping:
            node.arg = self.mapping[node.arg]
        return node

tree = ast.parse(minified_ws)
names_to_rename = set()

class Collector(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        names_to_rename.add(node.name)
        for a in node.args.args:
            names_to_rename.add(a.arg)
        self.generic_visit(node)
    def visit_ClassDef(self, node):
        names_to_rename.add(node.name)
        self.generic_visit(node)
    def visit_Assign(self, node):
        for t in node.targets:
            if isinstance(t, ast.Name):
                names_to_rename.add(t.id)
        self.generic_visit(node)
    def visit_For(self, node):
        if isinstance(node.target, ast.Name):
            names_to_rename.add(node.target.id)
        self.generic_visit(node)

Collector().visit(tree)
def is_valid_identifier(name: str) -> bool:
    return re.match(r"^[A-Za-z_][A-Za-z_0-9]*$", name) is not None

final_names = {n for n in names_to_rename if is_valid_identifier(n) and n not in SAFE_EXCLUSIONS}
mapping = {n: f"name_{n}" for n in sorted(final_names)}

renamer = Renamer(mapping)
new_tree = renamer.visit(tree)
ast.fix_missing_locations(new_tree)
clean_code = ast.unparse(new_tree) + "\n"

clean_txt_path = os.path.join(BASE, "clean.txt")
def to_visible_for_clean(txt: str) -> str:
    lines = txt.splitlines(keepends=True)
    out_lines = []
    for line in lines:
        vis = line
        if vis.endswith("\n"):
            vis = vis[:-1] + "\\n"
        out_lines.append(vis)
    return "\n".join(out_lines)

clean_visible = to_visible_for_clean(clean_code)
with open(clean_txt_path, "w", encoding="utf-8", newline="\n") as f:
    f.write(clean_visible)

clean_py_path = os.path.join(BASE, "clean.py")
def visible_to_python(txt: str) -> str:
    lines = txt.splitlines()
    real_lines = []
    for line in lines:
        if line.endswith("\\n"):
            content = line[:-2]
        else:
            content = line
        real_lines.append(content)
    return "\n".join(real_lines) + "\n"

with open(clean_py_path, "w", encoding="utf-8", newline="") as f:
    f.write(visible_to_python(clean_visible))

buf = io.StringIO()
with contextlib.redirect_stdout(buf):
    runpy.run_path(clean_py_path, run_name="__main__")
output = buf.getvalue()

raw_py_path, raw_txt_path, clean_txt_path, clean_py_path, output
